In [2]:
# So we can load files from other sub-directories, e.g. datasets.
import os
import sys

module_path = os.path.abspath(os.path.join('/u/scr/ananya/cifar_experiments/unlabeled_extrapolation'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
from torch import nn
import unlabeled_extrapolation.utils.utils as utils
import models
import json
import numpy as np
from scipy.special import softmax
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import preprocessing

import calibration as cal


In [3]:
print(f'cuda is available: {torch.cuda.is_available()}')
torch.cuda.empty_cache()

cuda is available: True


In [4]:
USE_CUDA = True
cuda = torch.device('cuda') 
batch_size, num_workers = 32, 16

In [5]:
def load_model(config_path, checkpoint_path):
    with open(config_path) as f:
        config = json.load(f)
    net = utils.initialize(config['model'])
    if USE_CUDA:
        net = net.cuda()
    net.new_last_layer(config['num_classes'])
    utils.load_ckp(checkpoint_path, net)
    return net

# Load datasets.
def load_test_dataset(config, idx, split_arg_name='split', split='val'):
    test_config = config['test_datasets'][idx]
    test_config['args'][split_arg_name] = split
    print(test_config['name'], test_config)
    if 'transforms' not in test_config:
        test_config['transforms'] = config['default_test_transforms']
    test_data = utils.init_dataset(test_config)
    test_loader = torch.utils.data.DataLoader(
        test_data, batch_size=batch_size,
        shuffle=False, num_workers=num_workers)
    return test_data, test_loader

def get_features_labels(net, loader):
    net.cuda()
    net.eval()
    feature_model = nn.Sequential(*list(net._model.children())[:-1])
    features_list, labels_list = [], []
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.cuda(), labels.cuda()
            features = feature_model(images)
            features_list.append(features.detach().cpu().numpy())
            labels_list.append(labels.detach().cpu().numpy())
    features = np.squeeze(np.concatenate(features_list))
    labels = np.concatenate(labels_list)
    return features, labels

def get_acc(preds, labels):
    return np.mean(preds == labels)

def make_none_list(rs, cs):
    return [[None] * cs for _ in range(rs)]

In [6]:
# Given a network how to get representations
def get_model_representations(
    config_paths, checkpoint_paths, model_names, loader_names, loader_indices, split_arg_names, split_names):
    M, L = len(model_names), len(loader_names)
    models = []
    with open(config_paths[0]) as f:
        config = json.load(f)
    for m in range(M):
        models.append(load_model(config_paths[m], checkpoint_paths[m]))

    loaders = []
    for l in range(L):
        _, loader = load_test_dataset(config, loader_indices[l], split_arg_names[l], split_names[l])
        loaders.append(loader)
    features, labels = make_none_list(M, L), make_none_list(M, L)
    for m in range(M):
        for l in range(L):
            features[m][l], labels[m][l] = get_features_labels(models[m], loaders[l])    
    return features, labels
        
def get_test_accs(features, labels, C=1.0):
    M, L = len(features), len(features[0])
    assert(L == 2)  # Only support train on first, validate on second for now
    clfs = []
    preds = []
    accs = []
    for m in range(M):
        cur_clf = LogisticRegression(random_state=0, C=C).fit(features[m][0], labels[m][0])
        cur_preds = cur_clf.predict(features[m][1])
        cur_acc = get_acc(cur_preds, labels[m][1])
        print(cur_acc)
        clfs.append(cur_clf)
        preds.append(cur_preds)
        accs.append(cur_acc)
    return clfs, preds, accs


In [7]:
# Living17 Swav-800

config_paths = [
    '/u/scr/ananya/cifar_experiments/unlabeled_extrapolation/logs/l17_swav_ft/augment_lr0.001/config.json',
]

checkpoint_paths = [
    '/u/scr/ananya/cifar_experiments/unlabeled_extrapolation/logs/l17_swav_ft/augment_lr0.001/checkpoints/ckp_0',
]

model_names = ['original_model']
loader_names = ['l17_trg_train', 'l17_trg_val', 'l17_src_train', 'l17_src_val']
split_names = ['train', 'val', 'train', 'val']
split_arg_names = ['split', 'split', 'split', 'split']
loader_indices = [1, 1, 0, 0]  # Relative to the config file in the first config path in config_paths.

swav_features, swav_labels = get_model_representations(
    config_paths, checkpoint_paths, model_names, loader_names, loader_indices,
    split_arg_names, split_names)

{'log_interval': 5000, 'use_cuda': True, 'save_freq': 1001, 'batch_size': 64, 'num_workers': 2, 'save_all_checkpoints': False, 'finetune': True, 'use_net_val_mode': True, 'optimizer': {'classname': 'torch.optim.SGD', 'args': {'lr': 0.001, 'momentum': 0.9}}, 'model': {'classname': 'models.imnet_resnet.ResNet50', 'args': {'pretrained': True, 'pretrain_style': 'swav', 'checkpoint_path': '/u/scr/ananya/simclr_weights/swav_800ep_pretrain.pth.tar'}}, 'criterion': {'classname': 'torch.nn.CrossEntropyLoss', 'args': {'reduction': 'mean'}}, 'root_prefix': '/scr/scr-with-most-space/', 'train_dataset': {'classname': 'datasets.breeds.Breeds', 'args': {'source': True, 'split': 'train', 'breeds_name': 'living17', 'root': '/scr/scr-with-most-space//imagenet/'}, 'transforms': [{'classname': 'torchvision.transforms.RandomResizedCrop', 'args': {'size': 224}}, {'classname': 'torchvision.transforms.RandomHorizontalFlip'}, {'classname': 'torchvision.transforms.ToTensor'}, {'classname': 'torchvision.transfor

In [59]:
# Entity-30 Swav-800

config_paths = [
    '/u/scr/ananya/cifar_experiments/unlabeled_extrapolation/logs/e30_ft_longer/augment_lr0.001/config.json',
]

checkpoint_paths = [
    '/u/scr/ananya/cifar_experiments/unlabeled_extrapolation/logs/e30_ft_longer/augment_lr0.001/checkpoints/ckp_0',
]

model_names = ['original_model']
loader_names = ['e30_trg_train', 'e30_trg_val', 'e30_src_train', 'e30_src_val']
split_names = ['train', 'val', 'train', 'val']
split_arg_names = ['split', 'split', 'split', 'split']
loader_indices = [1, 1, 0, 0]  # Relative to the config file in the first config path in config_paths.

swav_e30_features, swav_e30_labels = get_model_representations(
    config_paths, checkpoint_paths, model_names, loader_names, loader_indices,
    split_arg_names, split_names)

{'log_interval': 5000, 'use_cuda': True, 'save_freq': 1001, 'batch_size': 64, 'num_workers': 2, 'save_all_checkpoints': False, 'finetune': True, 'use_net_val_mode': True, 'optimizer': {'classname': 'torch.optim.SGD', 'args': {'lr': 0.001, 'momentum': 0.9}}, 'model': {'classname': 'models.imnet_resnet.ResNet50', 'args': {'pretrained': True, 'pretrain_style': 'swav', 'checkpoint_path': '/u/scr/ananya/simclr_weights/swav_800ep_pretrain.pth.tar'}}, 'criterion': {'classname': 'torch.nn.CrossEntropyLoss', 'args': {'reduction': 'mean'}}, 'root_prefix': '/scr/scr-with-most-space/', 'train_dataset': {'classname': 'datasets.breeds.Breeds', 'args': {'source': True, 'split': 'train', 'breeds_name': 'entity30', 'root': '/scr/scr-with-most-space//imagenet/'}, 'transforms': [{'classname': 'torchvision.transforms.RandomResizedCrop', 'args': {'size': 224}}, {'classname': 'torchvision.transforms.RandomHorizontalFlip'}, {'classname': 'torchvision.transforms.ToTensor'}, {'classname': 'torchvision.transfor

In [10]:
# Living17 MOCO

config_paths = [
    '../logs/breeds_moco_ft_0.003/config.json',
]

checkpoint_paths = [
    '../logs/breeds_moco_ft_0.003/checkpoints/ckp_0',
]

model_names = ['original_model']
loader_names = ['l17_trg_train', 'l17_trg_val', 'l17_src_train', 'l17_src_val']
split_names = ['train', 'val', 'train', 'val']
split_arg_names = ['split', 'split', 'split', 'split']
loader_indices = [1, 1, 0, 0]  # Relative to the config file in the first config path in config_paths.

features, labels = get_model_representations(
    config_paths, checkpoint_paths, model_names, loader_names, loader_indices,
    split_arg_names, split_names)

target_val_living {'name': 'target_val_living', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': False, 'split': 'train', 'breeds_name': 'living17', 'root': '/u/scr/nlp/imagenet/'}}
target_val_living {'name': 'target_val_living', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': False, 'split': 'val', 'breeds_name': 'living17', 'root': '/u/scr/nlp/imagenet/'}, 'transforms': [{'classname': 'torchvision.transforms.Resize', 'args': {'size': [224, 224]}}, {'classname': 'torchvision.transforms.ToTensor'}, {'classname': 'torchvision.transforms.Normalize', 'args': {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}}]}
source_val_living {'name': 'source_val_living', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': True, 'split': 'train', 'breeds_name': 'living17', 'root': '/u/scr/nlp/imagenet/'}}
source_val_living {'name': 'source_val_living', 'max_test_examples': 1000, 'classna

In [30]:
# Entity 30 MOCO

config_paths = [
    '../logs/entity30_moco_ft_0.003/config.json',
]

checkpoint_paths = [
    '../logs/entity30_moco_ft_0.003/checkpoints/ckp_0',
]

model_names = ['original_model']
loader_names = ['e30_trg_train', 'e30_trg_val', 'e30_src_train', 'e30_src_val']
split_names = ['train', 'val', 'train', 'val']
split_arg_names = ['split', 'split', 'split', 'split']
loader_indices = [1, 1, 0, 0]  # Relative to the config file in the first config path in config_paths.

e30_features, e30_labels = get_model_representations(
    config_paths, checkpoint_paths, model_names, loader_names, loader_indices,
    split_arg_names, split_names)

target_val_entity {'name': 'target_val_entity', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': False, 'split': 'train', 'breeds_name': 'entity30', 'root': '/u/scr/nlp/imagenet/'}}
target_val_entity {'name': 'target_val_entity', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': False, 'split': 'val', 'breeds_name': 'entity30', 'root': '/u/scr/nlp/imagenet/'}, 'transforms': [{'classname': 'torchvision.transforms.Resize', 'args': {'size': [224, 224]}}, {'classname': 'torchvision.transforms.ToTensor'}, {'classname': 'torchvision.transforms.Normalize', 'args': {'mean': [0.485, 0.456, 0.406], 'std': [0.229, 0.224, 0.225]}}]}
source_val_entity {'name': 'source_val_entity', 'max_test_examples': 1000, 'classname': 'datasets.breeds.Breeds', 'args': {'source': True, 'split': 'train', 'breeds_name': 'entity30', 'root': '/u/scr/nlp/imagenet/'}}
source_val_entity {'name': 'source_val_entity', 'max_test_examples': 1000, 'classna

In [ ]:
# # On CIFAR, Im-n-C10

# config_paths = [
#     '../logs/imnet_cifar_all_ft_0d01/config.json',
# ]

# checkpoint_paths = [
#     '../logs/imnet_cifar_all_ft_0d01/checkpoints/ckp_0',
# ]

# model_names = ['original_model']
# loader_names = ['e30_trg_train', 'e30_trg_val', 'e30_src_train', 'e30_src_val']
# split_names = ['train', 'val', 'train', 'val']
# split_arg_names = ['split', 'split', 'split', 'split']
# loader_indices = [4, 4, 0, 0]  # Relative to the config file in the first config path in config_paths.

# e30_features, e30_labels = get_model_representations(
#     config_paths, checkpoint_paths, model_names, loader_names, loader_indices,
#     split_arg_names, split_names)

In [20]:
np.std(features[0][2])

0.021199878

In [57]:
# Train logistic regression
def test_log_reg(features, labels, source_idx, test_indices, C=1.0, sgd=False):
    M, L = len(features), len(features[0])
    m = 0
    if not sgd:
        cur_clf = LogisticRegression(random_state=0, C=C).fit(features[m][source_idx], labels[m][source_idx])
    else:
        cur_clf = SGDClassifier(max_iter=10, loss='log', random_state=0, alpha=0.0, eta0=0.01, learning_rate='constant').fit(features[m][source_idx], labels[m][source_idx])
    for l in test_indices:
        cur_preds = cur_clf.predict(features[m][l])
        cur_acc = get_acc(cur_preds, labels[m][l])
        print(cur_acc)
    return cur_clf


def test_log_reg_warm_starting(features, labels, source_idx, test_indices, Cs=None):
    M, L = len(features), len(features[0])
    m = 0
    if Cs is None:
        Cs = np.logspace(-3, 2, 45)
    clf = LogisticRegression(random_state=0, warm_start=True, max_iter=1000)
    #.fit(features[m][source_idx], labels[m][source_idx])
    accs = []
    for C in Cs:
        clf.C = C
        clf.fit(features[m][source_idx], labels[m][source_idx])
        cur_accs = []
        for l in test_indices:
            cur_preds = clf.predict(features[m][l])
            cur_acc = get_acc(cur_preds, labels[m][l])
            cur_accs.append(cur_acc)
        print(cur_accs)
        accs.append(cur_accs)
    return clf, accs


In [ ]:
clf, accs = test_log_reg_warm_starting(swav_features, swav_labels, source_idx=2, test_indices=[1, 3])

[0.73, 0.9370588235294117]
[0.7441176470588236, 0.9411764705882353]
[0.7623529411764706, 0.9447058823529412]
[0.7741176470588236, 0.951764705882353]
[0.7811764705882352, 0.9552941176470588]
[0.7905882352941176, 0.9611764705882353]
[0.8, 0.9641176470588235]
[0.81, 0.9652941176470589]
[0.8117647058823529, 0.9652941176470589]
[0.81, 0.9641176470588235]
[0.8152941176470588, 0.9611764705882353]
[0.8129411764705883, 0.961764705882353]
[0.81, 0.961764705882353]
[0.8070588235294117, 0.9605882352941176]
[0.8029411764705883, 0.9594117647058824]
[0.7970588235294118, 0.96]
[0.7952941176470588, 0.96]
[0.7952941176470588, 0.96]


In [60]:
clf, accs = test_log_reg_warm_starting(swav_e30_features, swav_e30_labels, source_idx=2, test_indices=[1, 3])

[0.6376666666666667, 0.8498333333333333]
[0.6428333333333334, 0.8568333333333333]
[0.6456666666666667, 0.864]
[0.6478333333333334, 0.8701666666666666]
[0.648, 0.8763333333333333]
[0.6521666666666667, 0.8816666666666667]
[0.6523333333333333, 0.8881666666666667]
[0.654, 0.8918333333333334]
[0.6568333333333334, 0.8943333333333333]
[0.6578333333333334, 0.8986666666666666]
[0.6598333333333334, 0.9025]
[0.661, 0.9048333333333334]
[0.6608333333333334, 0.9068333333333334]
[0.6593333333333333, 0.9101666666666667]
[0.6566666666666666, 0.912]
[0.6556666666666666, 0.9148333333333334]
[0.6521666666666667, 0.917]
[0.651, 0.9183333333333333]
[0.647, 0.9193333333333333]
[0.6441666666666667, 0.9203333333333333]
[0.6396666666666667, 0.9218333333333333]
[0.6371666666666667, 0.9215]
[0.6346666666666667, 0.9201666666666667]
[0.6318333333333334, 0.9198333333333333]
[0.6268333333333334, 0.9183333333333333]
[0.6221666666666666, 0.9188333333333333]
[0.6166666666666667, 0.9191666666666667]
[0.6115, 0.9183333333

/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.591, 0.909]
[0.5871666666666666, 0.9066666666666666]
[0.5836666666666667, 0.9046666666666666]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.581, 0.9036666666666666]


...

[0.5616666666666666, 0.8938333333333334]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.5611666666666667, 0.8931666666666667]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[0.5601666666666667, 0.892]


In [61]:
print(accs)

[[0.6376666666666667, 0.8498333333333333], [0.6428333333333334, 0.8568333333333333], [0.6456666666666667, 0.864], [0.6478333333333334, 0.8701666666666666], [0.648, 0.8763333333333333], [0.6521666666666667, 0.8816666666666667], [0.6523333333333333, 0.8881666666666667], [0.654, 0.8918333333333334], [0.6568333333333334, 0.8943333333333333], [0.6578333333333334, 0.8986666666666666], [0.6598333333333334, 0.9025], [0.661, 0.9048333333333334], [0.6608333333333334, 0.9068333333333334], [0.6593333333333333, 0.9101666666666667], [0.6566666666666666, 0.912], [0.6556666666666666, 0.9148333333333334], [0.6521666666666667, 0.917], [0.651, 0.9183333333333333], [0.647, 0.9193333333333333], [0.6441666666666667, 0.9203333333333333], [0.6396666666666667, 0.9218333333333333], [0.6371666666666667, 0.9215], [0.6346666666666667, 0.9201666666666667], [0.6318333333333334, 0.9198333333333333], [0.6268333333333334, 0.9183333333333333], [0.6221666666666666, 0.9188333333333333], [0.6166666666666667, 0.919166666666

In [42]:
# Try normalizing.
mean, std = np.mean(swav_features[0][0], axis=0), np.std(swav_features[0][0], axis=0)
normalized_swav_features = []
for i in range(len(swav_features[0])):
    normalized_swav_features.append((swav_features[0][i] - mean) / std)
normalized_swav_features = [normalized_swav_features]

In [53]:
# Normalized features worked worse.
normalized_clf, normalized_accs = test_log_reg_warm_starting(normalized_swav_features, swav_labels, source_idx=2, test_indices=[1, 3])

[0.7811764705882352, 0.9588235294117647]
[0.7823529411764706, 0.9594117647058824]


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [54]:
# trg_clf = test_log_reg(features, labels, source_idx=0, test_indices=[1, 3], C=1.0)
src_clf = test_log_reg(features, labels, source_idx=2, test_indices=[1, 3], C=10.0, sgd=True)

0.7894117647058824
0.9317647058823529


/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:573: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


In [48]:
e30_trg_clf = test_log_reg(e30_features, e30_labels, source_idx=0, test_indices=[1, 3], C=1.0, sgd=True)
e30_src_clf = test_log_reg(e30_features, e30_labels, source_idx=2, test_indices=[1, 3], C=10.0, sgd=True)

KeyboardInterrupt: 

In [35]:
# Given two feature sets, make H-delta-H divergence dataset?
def make_hdeltah_labels(xs1, xs2):
    n1, n2 = len(xs1), len(xs2)
    ys = [0] * n1
    ys.extend([1] * n2)
    return np.concatenate([xs1, xs2]), np.array(ys)

def test_hdeltah(features, C=1.0):
    l17_hdeltah_xs_train, l17_hdeltah_ys_train = make_hdeltah_labels(features[0][2], features[0][0])
    l17_hdeltah_xs_val, l17_hdeltah_ys_val = make_hdeltah_labels(features[0][3], features[0][1])
    cur_clf = LogisticRegression(random_state=0, C=C).fit(l17_hdeltah_xs_train, l17_hdeltah_ys_train)
    cur_preds = cur_clf.predict(l17_hdeltah_xs_val)
    cur_acc = get_acc(cur_preds, l17_hdeltah_ys_val)
    return cur_clf, cur_acc
    

In [36]:
cur_clf, cur_acc = test_hdeltah(features, C=10.0)
print(cur_acc)

/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


NameError: name 'cur_avv' is not defined

In [38]:
cur_clf, cur_acc = test_hdeltah(e30_features, C=10.0)
print(cur_acc)

/juice/scr/ananya/cifar_experiments/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.804


In [39]:
import pickle
pickle.dump((features, labels), open("../logs/l17_moco_features_labels", "wb"))
pickle.dump((e30_features, e30_labels), open("../logs/e30_moco_features_labels", "wb"))

In [ ]:
features, labels = pickle.load(open("../logs/l17_moco_features_labels", "rb"))
e30_features, e30_labels = pickle.load(open("../logs/e30_moco_features_labels", "rb"))